In [63]:
#Importamos las librerias que vamos a usar
import pandas as pd
import numpy as np
import ast
import warnings
import nltk
import string
import re
warnings.filterwarnings("ignore")

In [64]:
#Creo una funcion de pasar el archivo de json a csv para no repetir codigo
def json_to_csv(archivo, archivo_final, coding):
    itemsList = []
# Abro un archivo y pongo el encoding que necesito
    with open(archivo, "r", encoding= coding) as f:
  # Recorro todo el json
        for line in f.readlines():
      # Todas las lineas del archivo las agrego a una lista
            itemsList.append(ast.literal_eval(line))
# Convierto la lista a un dataframe de pandas
        dfItems = pd.DataFrame(itemsList)
        #Devuelvo un csv para tener los archivos en un formato mas comodo y para optimizar los procesos de limpieza
        return dfItems.to_csv(archivo_final, index=False)

In [65]:
#Uso la funcion que creamos antes
json_to_csv("datasets/australian_user_reviews.json", "datasets/australian_user_reviews.csv", "utf-8")

In [66]:
#Uso la funcion que creamos antes
json_to_csv("datasets/australian_users_items.json", "datasets/australian_user_items.csv", "utf-8")

In [67]:
#Leemos el archivo de steam games
dfSteamGames= pd.read_json("datasets/output_steam_games.json", lines = True)

In [68]:
#Pasamos el archivo de steam a csv para leerlo mas facil
dfSteamGames.to_csv("datasets/output_steam_games.csv", index=False)

In [69]:
#Leemos los csv y los asignamos a estas variables
items = pd.read_csv("datasets/australian_user_items.csv")
reviews = pd.read_csv("datasets/australian_user_reviews.csv")
steam_games = pd.read_csv("datasets/output_steam_games.csv")

Archivo Steam Games

In [70]:
#Liempiza de datos del archivo Steam Games
#Seleccionamos las columnas que queremos eliminar que no sirven para el analisis
columnas_a_eliminar = ["app_name", "url", "tags", "reviews_url", "specs", "developer", "publisher", "early_access"]
#Eliminamos las columnas seleccionadas
steam_games.drop(columns=columnas_a_eliminar, inplace=True)

#Eliminamos las filas que tengan muchos datos nulos
steam_games = steam_games.dropna(thresh=4)

#Eliminamos los duplicados
steam_games.drop_duplicates(subset="id", inplace=True)

#Cambiamos la columna a tipo numero y los valores que sean letras que los ponga como nulos y los reemplazamos por 0
steam_games["price"] = pd.to_numeric(steam_games["price"], errors='coerce')
steam_games["price"] = steam_games["price"].fillna(0)

#Reemplazar los valores NaN por 0 para cambiar la columna a INT
steam_games["id"] = steam_games["id"].fillna(0)
#Cambio la columna a INT
steam_games["id"] = steam_games["id"].astype(int)

#Descomprimir la columna de genero("genres")
#Pasamos la columna de tipo objeto a tipo lista para poder usar el explode
from ast import literal_eval

def convertir_a_lista(cadena):
    try:
        return literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # Manejar el caso en el que la cadena no sea una lista válida

# Aplicar la función a la columna
steam_games["genres"] = steam_games["genres"].apply(convertir_a_lista)
#Aplicar el explode para descomprimir la lista
steam_games = steam_games.explode("genres")

#Reseteamos los indices
steam_games.reset_index(drop=True, inplace=True)

#Cambiamos la columna release date a tipo datetime
steam_games["release_date"] = pd.to_datetime(steam_games["release_date"],errors='coerce')

#Creamos una columna especifica para el año
steam_games["año"] = steam_games["release_date"].dt.year
#Usamos interpolacion linear para llenar los nulos de los años
steam_games["año"].interpolate(method='ffill', inplace=True)
#Pasamos la columna a tipo int
steam_games["año"] = steam_games["año"].astype(int)

#Eliminamos la columna release_date porque no la vamos a usar ya que tenemos el año
steam_games.drop(columns="release_date", inplace=True)

#Guardamos el csv limpio:
steam_games.to_csv("Csv_limpios/steam_games_limpio.csv", index=False)

Archivo User Items

In [71]:
#Limpieza de user items
#Eliminamos las columnas innecesarias
items = items.drop(columns=["user_url", "steam_id"])

# Aplicar la función que creamos antes para pasar la columna de objeto a lista y aplicar el pd.explode
items["items"] = items["items"].apply(convertir_a_lista)
#Aplicamos el explode
items = items.explode("items")

#Descomprimimos el diccionario de items en otro dataframe
items_inventario = pd.json_normalize(items["items"])

#Reseteamos los indices
items_inventario.reset_index(drop=True, inplace=True)
items.reset_index(drop=True, inplace=True)

#Juntamos los dataframes
items_final = items.join(items_inventario)

#Eliminamos la columna items que contiene el diccionario y no lo necesitamos
items_final.drop(columns="items", inplace=True)

#Cambiamos el tipo de dato de la columna de item id a INT
items_final["item_id"] = items_final["item_id"].fillna(0)
items_final["item_id"] = items_final["item_id"].astype(int)

#Guardamos el csv
items_final.to_csv("Csv_limpios/steam_items_limpio.csv", index=False)

Archivo User Reviews

In [72]:
reviews = pd.read_csv("datasets/australian_user_reviews.csv")

In [73]:
#Limpieza de reviews
#Eliminamos las columnas innecesarias
reviews = reviews.drop(columns="user_url")

#Eliminamos los duplicados
reviews.drop_duplicates(inplace=True)

# Aplicar la función que creamos antes para pasar la columna de objeto a lista y aplicar el pd.explode
reviews['reviews'] = reviews['reviews'].apply(convertir_a_lista)
#Aplicamos el explode
reviews = reviews.explode("reviews")

#Descomponemos el diccionario de la columna reviews
reviews_descomprimido = pd.json_normalize(reviews["reviews"])

#Eliminamos las columnas innecesarias
columnas = ["funny", "last_edited", "helpful"]
reviews_descomprimido.drop(columns=columnas, inplace=True)

#Eliminamos la columna reviews del dataframe original, para luego unirlo con el archivo reviews descomprimido
reviews.drop(columns="reviews", inplace=True)

#Cambiamos de la columna recommend los True por 1 y los False por 0
reviews_descomprimido["recommend"] = reviews_descomprimido["recommend"].replace(True, 1)
reviews_descomprimido["recommend"] = reviews_descomprimido["recommend"].replace(False, 0)
#Cambiamos la columna a tipo entero, eliminando los datos nulos que tiene la columna recommend
reviews_descomprimido.dropna(subset=["recommend"], inplace=True)
reviews_descomprimido["recommend"] = pd.to_numeric(reviews_descomprimido["recommend"], errors="coerce").astype(int)

#A reviews descomprimido le cambiamos el formato de la fecha:
reviews_descomprimido["posted"] = pd.to_datetime(reviews_descomprimido["posted"].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

#Usamos interpolacion linear para llenar los nulos de los años
reviews_descomprimido['posted'].interpolate(method='ffill', inplace=True)

#Creamos una columna especifica para el año:
reviews_descomprimido["year"] = reviews_descomprimido["posted"].dt.year.astype(int)

#Cambiamos la columna item_id a int
reviews_descomprimido["item_id"] = reviews_descomprimido["item_id"].astype(int)

#Eliminamos la columna posted ya que no la vamos a usar porque solo necesitamos el año
reviews_descomprimido.drop(columns="posted", inplace=True)


Modelo de NLP para crear la columna de sentiment_analysis

In [74]:
#Pasamos la columna a tipo string
reviews_descomprimido["review"] = reviews_descomprimido["review"].astype(str)
#A la columna de review le tenemos que sacar los signos de puntuacion y poner todo en minusculas para que el modelo
#Pueda funcionar de manera correcta
reviews_descomprimido["review"] = reviews_descomprimido["review"].str.lower()
reviews_descomprimido["review"] = reviews_descomprimido["review"].apply(lambda x: ''.join(char for char in x if char not in string.punctuation))
#Eliminamos los caracteres que no son alfabeticos
reviews_descomprimido["review"] = reviews_descomprimido["review"].apply(lambda x: re.sub("[^a-zA-Zñíá]", " ", str(x)))

In [75]:
#importamos lo necesario
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analizador_sentimientos = SentimentIntensityAnalyzer()

#Creamos una funcion que analiza los sentimientos del texto proporcionado 0(malo), 1(neutral), 2(positivo)
#Si es un dato nulo devuelve un 1 (neutral)
def sentimientos(texto):
    if pd.isnull(texto) or texto == "":
        return 1  # Valor neutral si el texto está vacío o es NaN
    elif isinstance(texto, str):
        # Realizamos análisis de sentimiento
        sentiment = analizador_sentimientos.polarity_scores(texto)
        compound_score = sentiment["compound"]
        if compound_score > 0:
            return 2 # Positivo
        elif compound_score < -0:
            return 0 # Negativo
        else:
            return 1 # Neutral
    else:
        return 1  # Valor neutral para datos que no son de tipo cadena
    
reviews_descomprimido["sentiment_analysis"] = reviews_descomprimido["review"].apply(sentimientos)

#Eliminamos la columna de reviews porque no la vamos a necesitar

reviews_descomprimido.drop(columns="review", inplace=True)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [76]:
#Reseteamos los indices para asegurarnos poder juntar correctamente los dataframes
reviews.reset_index(drop=True, inplace=True)
reviews_descomprimido.reset_index(drop=True, inplace=True)

#Unimos los dataframes de reviews y el de reviews_descomprimido y lo guardamos en un csv
reviews = reviews.join(reviews_descomprimido)

#Pasamos item id, recommend y sentiment analysis a entero
reviews.dropna(subset=["item_id"], inplace=True)
reviews["item_id"] = reviews["item_id"].astype(int)
reviews["recommend"] = reviews["recommend"].astype(int)
reviews["sentiment_analysis"] = reviews["sentiment_analysis"].astype(int)

#Guardamos el csv
reviews.to_csv("Csv_limpios/steam_reviews_limpio.csv", index=False)